In [7]:
import numpy as np
import torch
from torch import nn
import torchvision as tv
import matplotlib.pyplot as plt
%matplotlib inline

Get training data

In [15]:
path = "/vast/work/public/ml-datasets/mjsynth"
dataset = tv.datasets.ImageFolder(path, transform=tv.transforms)
#transform = transforms.Compose([transforms.Resize(255),transforms.CenterCrop(224),transforms.ToTensor()])

FileNotFoundError: Couldn't find any class folder in /vast/work/public/ml-datasets/mjsynth.

In [2]:
# print(images[0].shape)
batch_size = 32
inp_channels = 2
img_ht = 32
img_w = 16
out_ht = img_ht // 16 - 1
out_w = img_w // 4 -1

In [3]:
from collections import OrderedDict
class my_model(nn.Module):
  def __init__(self,img_channels,img_ht,img_w,num_class,map_to_seq_hidden = 64,rnn_hidden = 256):
    super(my_model, self).__init__()
    self.cnn_stack = nn.Sequential(
            OrderedDict([
              ('conv1', nn.Conv2d(img_channels,64,3,1,1)),
              # ('bn1',nn.BatchNorm2d(64)),
              ('relu1', nn.ReLU(inplace=True)),
              ('pool1',nn.MaxPool2d(kernel_size = 2,stride = 2)),

              ('conv2', nn.Conv2d(64,128,3,1,1)),
              # ('bn2',nn.BatchNorm2d(128)),
              ('relu2', nn.ReLU()),
              ('pool2',nn.MaxPool2d(kernel_size = 2,stride = 2)),

              ('conv3', nn.Conv2d(128,256,3,1,1)),
              # ('bn3',nn.BatchNorm2d(128)),
              ('relu3', nn.ReLU()),

              ('conv4', nn.Conv2d(256,256,3,1,1)),
              # ('bn4',nn.BatchNorm2d(256)),
              ('relu4', nn.ReLU()),
              ('pool4',nn.MaxPool2d(kernel_size = (1,2),stride = 2)),

              ('conv5', nn.Conv2d(256,512,3,1,1)),
              ('bn5',nn.BatchNorm2d(512)),

              ('conv6', nn.Conv2d(512,512,3,1,1)),
              ('bn6',nn.BatchNorm2d(512)),
              ('pool6',nn.MaxPool2d(kernel_size = (1,2),stride = 2)),

              ('conv7', nn.Conv2d(512,512,2,1,0)),
            ])
        )
    out_ht = img_ht // 16 - 1
    out_w = img_w // 4 -1
    self.map_to_seq = nn.Linear(512*out_ht, map_to_seq_hidden)
    self.lstm1 = nn.LSTM(map_to_seq_hidden, rnn_hidden, bidirectional=True)
    self.lstm2 = nn.LSTM(2 * rnn_hidden, rnn_hidden, bidirectional=True)
    self.out_ = nn.Linear(2 * rnn_hidden, num_class)

  def forward(self, x):
    conv = self.cnn_stack(x)
    batch, channel, height, width = conv.size()
    conv = conv.view(batch, channel * height, width)
    conv = conv.permute(2, 0, 1)  # (width, batch, feature)
    seq = self.map_to_seq(conv)
    lstm_, _ = self.lstm1(seq)
    lstm_, _ = self.lstm2(lstm_)
    out = self.out_(lstm_)
    return out

# net = my_model().cuda()
# Loss = torch.nn.CrossEntropyLoss()
# optimizer = torch.optim.SGD(net.parameters(), lr=0.01)

In [5]:
from torchsummary import summary

net = my_model(3,32,160,100).cuda()
summary(net,(3,32,160))

AttributeError: 'tuple' object has no attribute 'size'